# Nursery problem - Problema de la guardería

En la Europa de los 80's, más específicamente en 

La configuración inicial para trabajar en el entorno está dada por las siguientes líneas de código de importación de librerías y ubicación de nuestro DataSet.

In [2]:
# OS requirements
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in /home/alejocas/anaconda3/lib/python3.7/site-packages (2.3.0)


In [3]:
#ROOT_DIR = '/content/gdrive/'
#PERSONAL_DIR = ROOT_DIR + 'My Drive/'
#SHARED_UNITS_DIR = ROOT_DIR + 'Shared drives/'
#NURSERY_INFO_DIR = SHARED_UNITS_DIR + 'SimulaciónP/Nursery/'
#from google.colab import drive
#drive.mount(ROOT_DIR, force_remount=True)
# Importacion estandar de la libreria Pandas
# libreria manipulacion de datos
import os
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline


## Perfilamiento del DataSet
A continuación trataremos de explorar algo de información que pueda sernos de utilidad para abordar el problema con los modelos de ML adecuados.

In [4]:
CHARACTERISTIC_COLS = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
FULL_COLS = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'RESULT']
ndf = pd.read_csv('nursery.csv', delimiter=',', sep='\t', index_col=False) # Para que el dataset contenga las características y la variable objetivo
ndf.profile_report(style={'full_width':True})

In [78]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def oneHotEncoder(data):
    data_encoded = np.zeros((data.shape[0],0))
    if len(data.shape) == 1:
        num_of_cols = 1
    else:
        num_of_cols = data.shape[1]
    for col_index in range(num_of_cols):
        if num_of_cols == 1:
            values = array(data)
        else:
            values = array(data[:,col_index])
        values = values.reshape((-1,1))
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
        onehot_encoded = onehot_encoder.fit_transform(values)
        data_encoded = np.concatenate((data_encoded, onehot_encoded), axis = 1)
    return data_encoded

In [90]:
ndf = ndf[ndf['RESULT'] != 'recommend'] # Se remueve del conjunto de datos la categoría que solo tiene 2 muestras.
X_original = ndf.iloc[:, :-1].values
Y_original = ndf.iloc[:, 8].values
X_encoded = oneHotEncoder(X_original)
Y_encoded = oneHotEncoder(Y_original)

In [91]:
rows_length, cols_length = ndf.shape
print('Muestras:',rows_length, '- Variables/Atributos:', cols_length)

Muestras: 12958 - Variables/Atributos: 9


In [96]:
############## Plantilla de partición de datos ##################
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 3, shuffle = True)
for train_index, test_index in skf.split(X_encoded, Y_original):
    X_train, X_test = X_encoded[train_index], X_encoded[test_index]
    Y_train, Y_test = Y_original[train_index], Y_original[test_index]
    print("Train:", Y_train)
############## Plantilla de partición de datos ##################

Train: ['not_recom' 'priority' 'not_recom' ... 'spec_prior' 'spec_prior'
 'not_recom']
Train: ['priority' 'not_recom' 'priority' ... 'not_recom' 'not_recom'
 'spec_prior']
Train: ['priority' 'not_recom' 'priority' ... 'spec_prior' 'spec_prior'
 'not_recom']
